In [1]:
#!pip install openai
#!pip install dotenv

In [2]:
from typing import List
import os
import base64
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel

# Load environment variables
HOME_DIR = os.path.expanduser("~")
load_dotenv(f"{HOME_DIR}/.env")

# Step 1: Define row-oriented structure
class TableRow(BaseModel):
    Country: str
    OrgName: str
    OrgType: str
    Description: str
    Amount: str

class TableData(BaseModel):
    rows: List[TableRow]

# Step 2: OpenAI client
def get_openai_client() -> OpenAI:
    return OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Step 3: Encode image
def encode_image_to_base64(image_path: str) -> str:
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode("utf-8")

# Step 4: Prompt builder
def build_vision_prompt(prompt: str, base64_image: str) -> list:
    return [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{base64_image}"
                    }
                }
            ]
        }
    ]

# Step 5: Prompt text
def get_default_prompt() -> str:
    return """
        You are a data extraction assistant. Your task is to extract structured tabular data from an image of a table.
        Each row in the image has five fields:
            - Country
            - Partner Organization Name
            - Organization Type
            - Project Description
            - Amount (USD)

        Return a JSON array of dictionaries where each dictionary represents a row with these keys.
        Make sure all rows have the same structure. Fix OCR issues. If uncertain, mark data with [inferred].
        Do not guess numerical values. Only return the JSON in the correct format.
    """

# Step 6: Run GPT model and convert to DataFrame
def parse_table_data_from_image(image_path: str, model_name="gpt-4o") -> pd.DataFrame:
    client = get_openai_client()
    base64_img = encode_image_to_base64(image_path)
    prompt = get_default_prompt()
    messages = build_vision_prompt(prompt, base64_img)

    completion = client.beta.chat.completions.parse(
        model=model_name,
        messages=messages,
        temperature=0,
        response_format=TableData
    )

    return completion.choices[0].message.parsed 


In [14]:
i=4
for i in range(4,46):
    print(f'Error processing file: p{i}.png')
    try:
        image_path = f"../data/unfpa_partners/p{i}.png"
        response = parse_table_data_from_image(image_path)
        df = pd.DataFrame([row.model_dump() for row in response.rows])
    
        df.to_csv(f'../data/unfpa_partners/p{i}.csv', index=False)

    except:
        print(f'Error processing file: p{i}.png')
        continue
    

Error processing file: p4.png
Error processing file: p5.png
Error processing file: p6.png
Error processing file: p7.png
Error processing file: p8.png
Error processing file: p9.png
Error processing file: p10.png
Error processing file: p11.png
Error processing file: p12.png
Error processing file: p13.png
Error processing file: p14.png
Error processing file: p15.png
Error processing file: p16.png
Error processing file: p17.png
Error processing file: p18.png
Error processing file: p19.png
Error processing file: p20.png
Error processing file: p21.png
Error processing file: p22.png
Error processing file: p23.png
Error processing file: p24.png
Error processing file: p25.png
Error processing file: p26.png
Error processing file: p27.png
Error processing file: p28.png
Error processing file: p29.png
Error processing file: p30.png
Error processing file: p31.png
Error processing file: p32.png
Error processing file: p33.png
Error processing file: p34.png
Error processing file: p35.png
Error processi

In [15]:
datas = []
for i in range(1,46):
    
    df = pd.read_csv(f'../data/unfpa_partners/p{i}.csv')
    datas.append(df)
data = pd.concat(datas)


In [16]:
data.to_csv(f'../data/unfpa_partners.csv', index=False)

In [13]:
def get_county_gpt(cities_states):

    prompt = """
    You're an expert in US geography, You know all cities, counties and states. 
    I will provide you with the City and State (two letter code) and I want you to tell me the County.
    Please give me only the County name, nowthing else (Do not write a sentence). 
    If the City is an independent city, just return the name of the city. 
    If you can't find the county, return  the City name provided.
    Make sure to check the accuracy of your answer from reliable sources, only
    provide and answer if you're 100% sure. 
    
    Example of acceptable results:
      Elysian Fields
      Leon
      Prince George's
      Salt Lake
      Virginia Beach
      
    Example of unacceptable results:
      Elysian Fields is located in Harrison County.
      Leon County
      Virginia Beach is an independent city and not part of any county.
      I\'m sorry, but there doesn\'t appear to be a "South Ho, IL" in Illinois. Could you please check the spelling or provide more details?
      
    Here is the City, State:
    {INPUT}
    """
    
    #cities_states=['Bridgton, ME','Brewer, ME','Tallahassee, FL']
    counties_list = []
    for name in tqdm(cities_states):
        resp = get_completion(prompt.format(INPUT=name), model="gpt-4o", temp=0)
        counties_list.append(resp)
    return counties_list

4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
